In [2]:
import pandas as pd
import numpy as np

# from datetime import datetime

import warnings
warnings.filterwarnings('ignore', message='FixedFormatter')

respath = '../mDescr/'
date = '140423'# datetime.today().strftime('%d%m%y')

In [481]:
def read_res_sheets(cohort, res_file = 'Descriptives', date='140423', v=True):
    file = '../'+cohort+'/Descriptives/'+cohort+'_'+res_file+'_'+date+'.xlsx'
    xl = pd.ExcelFile(file)
    for sheet in xl.sheet_names:
        if v: print(cohort+'_'+sheet, end=', ')
        df = xl.parse(sheet) # convert to dataframe
        globals()[cohort+'_'+sheet] = df

read_res_sheets('GENR')
read_res_sheets('ALSPAC')

GENR_missing_pattern, GENR_summary, GENR_s_imp_cnt, GENR_s_imp_cat, GENR_cor_imp, ALSPAC_missing_pattern, ALSPAC_s_pre_imp, ALSPAC_s_imp_cnt, ALSPAC_s_imp_cat, ALSPAC_cor_imp, 

In [478]:
ns = {'exercise':'exerc','sleep_hr':'sleep','med_diet':'mdiet', 'sleep_SR':'SRsleep',
      'exercise_bin':'exerc_bin','sleep_hr_bin':'sleep_bin','med_diet_bin':'mdiet_bin', 'sleep_SR_bin':'SRsleep_bin',
      'exercise_age':'exerc_age','sleep_hr_age':'sleep_age','med_diet_age':'mdiet_age',
      'intern_score_13':'intern', 'tot_fat_percent_13':'adipos', 'risk_groups':'comorb', 
      'pre_life_events':'pre_le', 'pre_contextual_risk':'pre_cr', 'pre_parental_risk':'pre_pr', 'pre_interpersonal_risk':'pre_ir', 
      'post_life_events':'pos_le','post_contextual_risk':'pos_cr','post_parental_risk':'pos_pr','post_interpersonal_risk':'pos_ir', 
      'post_direct_victimization':'pos_dv'}

In [482]:
def desc(sample = 'GENR_', varz=['age_child','sex','ethnicity','ELS','prenatal_stress','postnatal_stress',
                                 'exercise','sleep_hr','med_diet',
                                 'exercise_age','med_diet_age','sleep_hr_age',
                                 'intern_score_13','tot_fat_percent_13','risk_groups',
                                 'm_bmi_before_pregnancy','m_smoking','m_drinking',
                                 'exercise_bin','sleep_hr_bin','med_diet_bin',
                                 # 'intern_score_09','tot_fat_percent_09',
                                 'parity','gest_age_birth','gest_weight'], 
         varz_cat=['sex','ethnicity','exercise','risk_groups','m_smoking', # 'm_drinking',
                  'exercise_bin','sleep_hr_bin','med_diet_bin']):

    def prep(df):
        df["group_no"] = df.isnull().all(axis=1).cumsum() # assign new number every empty row
        df = df.set_index('group_no').dropna(how='all') 
        out = pd.DataFrame()
        for i in df.index.unique():
            name = df.loc[i,df.columns[0]].iloc[0]
            df.rename(index={i: name}, inplace=True)
            out = out.append(df.loc[name,].iloc[1:])
        return(out)


    preimp = prep(globals()[f'{sample}s_pre_imp'].copy())
    preimp = preimp.loc[preimp.value!='SD',]

    poscat = prep(globals()[f'{sample}s_imp_cat'].copy())
    poscat.loc['comorb','row.names(v_summ)'] = ['healthy', 'high_intern','high_adipos','comorbidity']

    posimp = globals()[f'{sample}s_imp_cnt'].set_index(
        'c("Min", "1stQ", "Median", "Mean", "3rdQ", "Max", "SD")').T
    
    missdf = globals()[f'{sample}missing_pattern'].set_index('Var name')

    def mean_range(var, varpos=''):
        prel1,prem,prel2 = preimp.loc[preimp['value'].str.contains('Min|Mean|Max', na=False),].loc[var,'sample']
        posl1,posm,posl2 = posimp.loc[varpos][['Min','Mean','Max']]

        return f'{float(prem):.2f} ({float(prel1):.2f}-{float(prel2):.2f}) \t{float(posm):.2f} ({float(posl1):.2f}-{float(posl2):.2f})'
    
    def n_perc(var, varpos=''):
        ns = preimp.loc[var,['value','sample']].set_index('value') #.drop(index='SD')
        ns['pre']=ns['sample'].astype(str)+' ('+(ns['sample'].astype(float)/ns['sample'].astype(float).sum()*100).round(2).astype(str)+'%)'
        ns['pos']=np.nan
        if varpos in poscat.index:
            ns2 = poscat.loc[varpos].set_index(poscat.columns[0])
            ns.loc[:-1,'pos']=ns2['counts'].astype(float).astype(int).astype(str)+' ('+ns2['percent'].astype(float).round(2).astype(str)+'%)'
        return ns

    out = pd.DataFrame()
    
    for var in varz: 
        vp = ns[var] if var in ns.keys() else var
        if var in varz_cat:
            nl = n_perc(var, vp)
            nl = nl.set_index(var+' '+nl.index.astype(str))
        else:
            nl = pd.DataFrame([mean_range(var, vp)], index=[var])
            nl[['pre', 'pos']] = nl.iloc[:,0].str.split('\t', 1, expand=True)
            
        nl['miss'] = missdf.loc[var, missdf.columns[1]]
        out = pd.concat([out,nl[['pre','miss','pos']]])

    return(out)



In [483]:
t = desc('ALSPAC_')
t.to_csv('~/Desktop/T.csv')